# Gerador de lero-lero usando uma rede neural

Feito por Erick Gaiote.

Esse documento é um relato da minha tentativa de criar um gerador de lero-lero usando algumas técnicas aprendidas no curso de Computação
Científica e Análise de Dados da UFRJ no período 2023.2. Um gerador de lero-lero é um programa que gera texto de forma automática e aleatória,
geralmente com o uso de inteligência artificial.

## Entendendo predição de palavras

Para criar o gerador, pensei em um problema mais simples. Imagine que um usuário está digitando um texto, e então para, sem terminá-lo. Qual é a próxima palavra que ele digitará? Em outras palavras, qual seria a próxima palavra a ser digitada baseada no *contexto* anterior. Esse era o problema original que eu queria resolver, mas percebi que, resolvendo esse problema, seria possível criar um gerador de lero-lero. Se o programa predissesse a próxima palavra a ser digitada, ele poderia incluí-la no contexto para gerar a palavra seguinte, e repetir o processo até gerar um texto completo. Seria necessário, é claro, gerar uma palavra com um contexto vazio para escolher a primeira palavra, e mantive isso em mente também.

Esse problema me levou a pesquisar dois modelos muito importantes na modelagem de texto: o *Skip-gram* e o *CBOW* (*Continuous Bag Of Words*).

### Skip-gram

O Skip-gram é um modelo usado para, dada uma palavra, predizer o contexto em volta dela. Em outras palavras, ele resolve o problema de adivinhar as palavras que aparecem em volta de uma determinada palavra em um texto.

![](https://1.bp.blogspot.com/-Vz5pLuZ49K8/XV0ErlMtdDI/AAAAAAAAB0A/FIM74z__LAUkCqpW12ViAnGX8Br56W2PQCEwYBhgL/s1600/image001.png)

Imagem [dessa publicação](https://thinkinfi.com/word2vec-skip-gram-explained/) do Think Infi.

Mas esse não é o problema que queremos resolver. Embora seja útil gerar várias das próximas palavras a partir da última palavra, não pareceu a abordagem ideal. Isso porque, nesse modelo, você insere apenas uma palavra. Mas prever todo o contexto baseado em uma única palavra para gerar algo do zero parece radical demais. Me pareceu mais razoável gerar as próximas palavras baseado em várias palavras anteriores.

E é justamente isso que o modelo CBOW faz.

### CBOW

O CBOW é um modelo usado para, dado um contexto em volta de uma palavra desconhecido, predizer essa palavra. Em outras palavras, ele resolve o problema de adivinhar uma palavra dadas as palavras que aparecem em volta dela em um texto.

![](https://media.geeksforgeeks.org/wp-content/uploads/20230416175135/Screenshot-(26).png)

Imagem [dessa publicação](https://www.geeksforgeeks.org/continuous-bag-of-words-cbow-in-nlp/) do GeeksforGeeks.

Esse é o problema que queremos resolver, com uma pequena diferença. Geralmente, o CBOW leva em consideração as palavras que ocorrem antes *e* depois de uma palavra. Mas nós ainda não sabemos o que virá depois da palavra que queremos predizer. Seria possível fazer esse modelo funcionar utilizando apenas as palavras anteriores?

Assim, pesquisei para saber como esse modelo é usado, para ver se é possível fazer essa modificação de forma que ele funcionasse.

### Funcionamento dos modelos

Ambos os modelos anteriores são treinados usando *redes neurais*. Explicarei como elas funcionam posteriormente. Por enquanto, veja elas como uma caixa preta que recebe como entrada uma sequência de um tamanho fixo de números e entrega como saída uma predição que é uma sequência, também de tamanho fixo, de números. Você também pode treinar essa caixa preta dando vários exemplos de entrada e saídas esperadas para essas entradas, para que ela "aprenda" a prever os resultados para entradas que ela nunca viu antes.

Então, para fazer um modelo CBOW predizer uma palavra, precisamos treinar a rede neural com vários exemplos de contexto (entrada) para palavra (saída esperada). Mas tem um problema: uma rede neural recebe apenas números. Isso acontece por causa da natureza aritmética das operações que são feitas em uma rede neural. Não é possível somar, multiplicar ou calcular o logarítmo de palavras. A não ser que você as modele.

#### Modelando palavras

Precisamos encontrar uma maneira de transoformar palavras em números. A primeira e mais óbvia é atribuir um número para cada palavra. Por exemplo, a palavra "cachorro" pode ser 1 e "chocolate" pode ser 2. Isso pode ser simples, mas traz alguns problemas. Se "cachorro" é 1 e "chocolate" é 2, então um cachorro é meio chocolate? Se "restaurante" é 3, então um restaurante é um cachorro mais um chocolate? Quando tratamos da semântica das palavras, essa abordagem pressupõe uma sequencialidade dos significados que é bem inconveniente, espacialmente quando serão feitas operações aritméticas com os números que identificam esses significados.

O que é feito em vez disso é usar a codificação *one-hot*. A codificação one-hot é uma forma de modelar ocorrências de determinadas categorias dentre uma lista fixa de categorias usando vetores. Cada posição de um vetor one-hot corresponde a uma categoria, e possui o valor 1 se o vetor representa aquela categoria e 0 caso contrário.

![](https://miro.medium.com/v2/resize:fit:674/1*9ZuDXoc2ek-GfHE2esty5A.png)

Imagem [dessa publicação](https://medium.com/intelligentmachines/word-embedding-and-one-hot-encoding-ad17b4bbe111) do Medium.

No nosso caso, cada palavra existente — ou cada palavra em um vocabulário delimitado — será uma categoria, e terá uma posição específica no vetor. Por exemplo, se estamos trabalhando com 1000 palavras diferentes e "garrafa" é a palavra número 50, o vetor que representa essa palavra é

$$
v(\text{garrafa}) = (\underbrace{0, ..., 0}_{\text{49 vezes}}, 1, \underbrace{0, ..., 0}_{\text{950 vezes}})
$$

Isso elimina os problemas semânticos que teríamos no modelo anterior, agora que cada palavra está confinada à sua própria dimensão no vetor. Esse modelo também é útil para representar as probabilidades de uma palavra desconhecido ser cada palavra do vocabulário, como veremos mais à frente.

#### Especificando e usando a rede neural

Usando a codificação one-hot, podemos desenhar o formato que a entrada e a saída da rede neural precisam ter. Precisamos também definir o tamanho do vocabulário e quantas palavras queremos usar para formar os contextos. Digamos que temos $N$ palavras no vocabulário e o contexto possui $T$ palavras (contando o contexto anterior e posterior à palavra). No caso do CBOW, podemos ter como entrada $T$ vetores de $N$ espaços, cada um representando uma palavra do contexto, e, como saída, um único vetor de $N$ espaços, representando a palavra predita. Note que a ordem dos vetores de entrada importa: as palavras imediatamente próximas da palavra a ser predita podem ter mais peso para determiná-la do que as palavras mais distantes. Podemos também, para facilitar, representar a entrada como um único vetor de $T \cdot N$ posições.

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*cuOmGT7NevP9oJFJfVpRKA.png)

Imagem [dessa publicação](https://towardsdatascience.com/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314) do Medium.

A saída da rede neural não será, realmente, o vetor one-hot que representa a palavra predita. Tal exatidão dificilmente é dada por redes neurais. Em vez disso, podemos ter, para cada palavra do vocabulário, a *probabilidade* da palavra predita ser essa palavra. Isso dá, inclusive, uma característica interessante ao gerador de lero-lero. Se tivéssemos simplesmente o vetor one-hot da próxima palavra, geraríamos sempre o mesmo texto. Ora, se tivermos as probabilidades de cada palavra, podemos sortear a próxima palavra de acordo com a probabilidade dada pela rede neural. Esse, pelo menos, me pareceu um bom plano.

#### Adaptando o modelo

Mas, no fim, podemos adaptar o modelo para usar apenas o contexto anterior à palavra predita? A resposta é que, para os nossos propósitos, sim. Basta, ao definir a entrada da rede neural, considerar apenas os vetores das palavras anteriores. Não precisamos nos preocupar em isso deixar nosso modelo impreciso, porque nem teremos as palavras posteriores a princípio.

Mas, então, por que o modelo original inclui as palavras posteriores, e por que podemos descartar isso? Ambos o CBOW e o Skip-gram são modelos usados para gerar representações vetoriais de palavras de forma a manter palavras com semânticas relacionadas próximas no espaço vetorial. O propósito direto da rede neural *é* prever palavras, mas, como resultado, ela produz esses vetores na forma dos pesos que a rede neural aprende a dar a cada posição da entrada (falaremos mais sobre isso adiante). Nesse caso, as palavras usadas após uma determinada palavra também são úteis para entender quais palavras são usadas em contextos parecidos. No nosso caso, porém, como não estamos interessados nessas representações vetoriais, isso pode ser descartado. Estamos usando o modelo apenas para prever as palavras.

## Entendendo redes neurais

Meu melhor amigo nesse trabalho de entender redes neurais foi a playlist sobre redes neurais do 3Blue1Brown, que pode ser acessada [aqui](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi). Nela, ele explica de forma didática, e bem mais visual do que mostrarei aqui, como redes neurais funcionam. Ele também aborda o tema de uma forma bem mais profunda do que abordarei, então definitivamente vale a pena ver depois de ler esse notebook. De qualquer forma, continuemos.

### Estrutura

Uma rede neural é um modelo computacional capaz de fazer predições e reconhecer padrões utilizando como base um conjunto de dados de treinamento, o que constitui o famoso Aprendizado de Máquina. Uma rede neural é composta pelo menos duas *camadas*: uma camada de entrada e uma camada de saída. Uma *camada* é composta por uma quantidade fixa de *neurônios*. Um *neurônio* é simplesmente uma variável numérica.

Podemos modelar, matematicamente, cada camada como um vetor. Seja a camada de entrada um vetor de $I$ espaços

$$
i =
\begin{bmatrix}
i_1    \\
i_2    \\
\vdots \\
i_I
\end{bmatrix}
$$

e a camada de saída um vetor de $O$ espaços

$$
o =
\begin{bmatrix}
o_1    \\
o_2    \\
\vdots \\
o_I
\end{bmatrix}
$$

Também são postas uma série de camadas entre essas duas, que são as chamadas camadas *ocultas*. Cada camada oculta pode ser representada por um vetor de $H^{(k)}$ espaços

$$
h^{(k)} =
\begin{bmatrix}
h^{(k)}_1    \\
h^{(k)}_2    \\
\vdots \\
h^{(k)}_{H^{(k)}}
\end{bmatrix}
$$

com $k = 1,\dots,\bar{H}$, sendo $\bar{H}$ o número de camadas ocultas da rede.

### Computando predições

Além disso, entre uma camada e outra, há uma aresta de cada neurônio da camada anterior para cada neurônio da camada seguinte. Esse aresta tem um peso, que servirá para calcular os valores da camada seguinte. Seja $W^{(k)}_{ij}$, o peso da aresta do neurônio $i$ da $(k-1)$ª camada para o neurônio $j$ da $k$ª camada. Esses pesos, inicialmente, são gerados aleatoriamente.

Sejam $l^{(1)} = i, l^{(2)} = h^{(1)}, \dots, l^{(N - 1)} = h^{(\bar{H})}, l^{(N)} = o$ todas as camadas da rede, sendo $N = \bar{H} + 2$ o número de camadas. Seja também $L^{(k)}$ o número de posições de $l^{(k)}$, para $k = 1,\dots,N$. Nesse caso, para todo $k = 2,\dots,N$, e todo $j = 1,\dots,L^{(k)}$, temos

$$
l^{(k)}_j = \sum_{i = 1}^{L^{(k - 1)}} W^{(k)}_{ij} \cdot l^{(k - 1)}_i =
\begin{bmatrix}
W^{(k)}_{1j} & \dots & W^{(k)}_{L^{(k - 1)}}
\end{bmatrix}
\begin{bmatrix}
l^{(k - 1)}_1             \\
\vdots                    \\
l^{(k - 1)}_{L^{(k - 1)}}
\end{bmatrix} =
\begin{bmatrix}
W^{(k)}_{1j} & \dots & W^{(k)}_{L^{(k - 1)}}
\end{bmatrix} l^{(k - 1)}
$$

Isso sugere que podemos interpretar $W^{(k)}$, para $k = 2,\dots,N$ como uma matriz $L^{(k - 1)} \times L^{(k)}$ tal que

$$
W =
\begin{bmatrix}
w_{11}           & \dots  & w_{1L^{(k)}}           \\
\vdots           & \ddots & \vdots                 \\
w_{L^{(k - 1)}1} & \dots  & w_{L^{(k - 1)}L^{(k)}}
\end{bmatrix}
$$

Assim, podemos escrever que, para $k = 2,\dots,N$, temos

$$
l^{(k)} = W^{(k)} \cdot l^{(k - 1)}
$$

Em outras palavras, temos que cada camada é uma transformação linear da camada anterior, e cada camada após a primeira possui sua própria
transformação linear. Mas não é tão simples. Na verdade, cada camada após a primeira possui uma função chamada *função de ativação*, que é uma
função aplicada em cada elemento da camada após a transformação acontecer. Seja $a^{(k)}$ para $k = 2,\dots,N$ a função de ativação para a $k$ª
camada. Então

$$
l^{(k)} = a^{(k)}(W^{(k)} \cdot l^{(k - 1)}) =
\begin{bmatrix}
a^{(k)} \left( \left( W^{(k)} \cdot l^{(k - 1)} \right)_1 \right)         \\
\vdots                                                                    \\
a^{(k)} \left( \left( W^{(k)} \cdot l^{(k - 1)} \right)_{L^{(k)}} \right)
\end{bmatrix}
$$

Há um grande conjunto de funções de ativação que são usadas dependendo do problema que o modelo pretende resolver. Algumas delas podem ser
vistas [aqui](https://en.wikipedia.org/wiki/Activation_function#Table_of_activation_functions). Dentre as mais famosas, estão a função
identidade

$$
f(x) = x
$$

A função unidade linear retificada (ReLU, Rectified Linear Unit)

$$
(x)^+ = \max{\{0, x\}}, \, \, ((x_1, x_2, \dots))^+ = ((x_1)^+, (x_2)^+, \dots)
$$

A função sigmoide

$$
\sigma(x) = \frac{1}{1 + e^{-x}}, \, \, \sigma((x_1, x_2, \dots)) = (\sigma(x_1), \sigma(x_2), \dots)
$$

E a função softmax

$$
\text{softmax}(x_{T \times 1}) = \text{softmax}((x_1, \dots, x_T)) =
\left( \frac{e^{x_1}}{\sum_{i = 1}^T e^{x_i}}, \dots, \frac{e^{x_T}}{\sum_{i = 1}^T e^{x_i}} \right)
$$

Temos então um algoritmo para computar a saída de uma rede neural para uma determinada entrada.

$
\text{Função Predizer}(i)                                            \\
\; \; \; \; \; \; \; \text{Para } k \text{ de } 2 \text{ até } N     \\
\; \; \; \; \; \; \; \; \; \; \; \; i \gets a^{(k)}(W^{(k)} \cdot i) \\
\; \; \; \; \; \; \; \text{Retorne } i
$

Eu mencionei que os valores de $W^{(k)}$ para $k = 2,\dots,N$ são iniciados aleatoriamente, e isso faz com que a predição dada não seja boa.
Como podemos, então, fazer com que a rede neural aprenda para dar predições melhores?

### Aprendizado de Máquina

Uma rede neural "aprende" a fazer predições melhores ao ser alimentada com um conjunto de dados de treinamento. Esse conjunto de dados possui um
número grande de exemplos, cada exemplo sendo constituído por uma entrada e uma saída esperada para essa entrada. A rede neural computa as
prediões para a entrada de cada exemplo e compara com a saída esperada usando uma *função erro*. Uma *função erro* é uma função que quantifica,
dada a predição e a saída esperada para uma determinada entrada, o quanto a predição difere da saída esperada. A média da função erro dá o erro
da rede inteira. Existem várias funções erro. Dentre as mais famosas, estão o erro quadrático:

$$
E(x, y) = (x - y)^2
$$

E a entropia cruzada:

$$
E(x_{T \times 1}, y_{T \times 1}) = - y_i \ln{x_i}
$$

Vamos formalizar isso. Seja $M \in \mathbb{N}$ o número de exemplos, e sejam os vetores $p_i$ e $o_i$ com $T$ posições, respectivamente, a
predição, e a saída esperada de cada exemplo, para $i = 1,\dots,M$. E seja $E(x, y)$ a função erro. Temos que o erro total da rede é dado por

$$
E_{\text{T}} = \frac{1}{M} \sum_{m = 1}^{M} \sum_{t = 1}^{T} E((p_m)_t, (o_m)_t)
$$

Para fazer uma predição boa, queremos minimizar esse valor em função de todos os pesos da rede neural. Isso porque queremos ajustar os pesos,
que geramos aleatoriamente, para melhorar as predições. Então, na realidade, temos uma função de *muitas* variáveis

$$
E_{\text{T}}(W^{(2)}_{11}, \dots, W^{(2)}_{L^{(1)}L^{(2)}}, \dots, W^{(N)}_{11}, \dots, W^{(N)}_{L^{(N - 1)}L^{(N)}})
$$

E como podemos minimizar essa função? Podemos usar o algoritmo do Gradiente Descendente para encontrar um mínimo local da função. Mas, para isso,
precisamos calcular o gradiente da função:

$$
\nabla E_{\text{T}} =
\begin{bmatrix}
\frac{\delta E_{\text{T}}}{\delta W^{(2)}_{11}}               \\
\vdots                                                        \\
\frac{\delta E_{\text{T}}}{\delta W^{(N)}_{L^{(N-1)}L^{(N)}}}
\end{bmatrix}
$$

#### Backpropagation

Para calcular todas as derivadas, vamos usar Backpropagation. Vamos focar em uma única tripla $i, j, k$ para a qual queremos calcular a derivada

$$
\frac{\delta E_{\text{T}}}{\delta W^{(k)}_{ij}}
= \frac{\delta}{\delta W^{(k)}_{ij}} \frac{1}{M} \sum_{m = 1}^{M} \sum_{t = 1}^{T} E
= \frac{1}{M} \sum_{m = 1}^{M} \sum_{t = 1}^{T} \frac{\delta E}{\delta W^{(k)}_{ij}}
$$

$E$ é uma função de $(o_m)_t$, que é uma constante, e de $(p_m)_t$, que é função de $W^{(k)}_{ij}$, então podemos usar a Regra da Cadeia:

$$
\frac{1}{M} \sum_{m = 1}^{M} \sum_{t = 1}^{T} \frac{\delta E}{\delta W^{(k)}_{ij}}
= \frac{1}{M} \sum_{m = 1}^{M} \sum_{t = 1}^{T} \frac{\delta E}{\delta (p_m)_t} \frac{\delta (p_m)_t}{\delta W^{(k)}_{ij}}
$$

Mas $(p_m)_t = l^{(N)}_t$, que é função de $i_m$ (para não estranhar o sumiço do $m$) e de $W^{(k)}_{ij}$, então podemos escrever como:

$$
\frac{\delta (p_m)_t}{\delta W^{(k)}_{ij}} = \frac{\delta l^{(N)}_t}{\delta W^{(k)}_{ij}}
= \frac{\delta}{\delta W^{(k)}_{ij}} (a^{(N)}(W^{(N)} \cdot l^{(N - 1)}))_t
= \frac{\delta}{\delta W^{(k)}_{ij}} a^{(N)}((W^{(N)} \cdot l^{(N - 1)})_t)
= \frac{\delta}{\delta W^{(k)}_{ij}} a^{(N)} \left( \underbrace{\sum_{q = 1}^{L^{(N - 1)}} W^{(N)}_{qt} l^{(N - 1)}_q}_u \right) \\
= \frac{\delta a^{(N)}}{\delta u} \frac{\delta}{\delta W^{(k)}_{ij}} \sum_{q = 1}^{L^{(N - 1)}} W^{(N)}_{qt} l^{(N - 1)}_q
= \frac{\delta a^{(N)}}{\delta u} \sum_{q = 1}^{L^{(N - 1)}} \frac{\delta}{\delta W^{(k)}_{ij}} (W^{(N)}_{qt} l^{(N - 1)}_q)
= \frac{\delta a^{(N)}}{\delta u} \sum_{q = 1}^{L^{(N - 1)}}
  \left( \frac{\delta W^{(N)}_{qt}}{\delta W^{(k)}_{ij}} l^{(N - 1)}_q + \frac{\delta l^{(N - 1)}}{\delta W^{(k)}_{ij}} W^{(N)}_{qt} \right)
$$

$\frac{\delta W^{(N)}_{qt}}{\delta W^{(k)}_{ij}}$ é fácil de calcular. Se $q = i$, $t = j$ e $N = k$, então
$\frac{\delta W^{(N)}_{qt}}{\delta W^{(k)}_{ij}} = 1$, senão $\frac{\delta W^{(N)}_{qt}}{\delta W^{(k)}_{ij}} = 0$. Isso acontece porque os pesos
não são funções de outros pesos.

Já $\frac{\delta l^{(N - 1)}}{\delta W^{(k)}_{ij}}$ pode ser calculado da mesma forma que calculamos $\frac{\delta l^{(N)}}{\delta W^{(k)}_{ij}}$.
Podemos repetir esse processo para todas as camadas até a segunda (a primeira é constante).

Para fazer esse cálculo, então, só é preciso conhecer a derivada de $E$ e a derivada de cada função de ativação $a^{(k)}$ para cada
$k = 2,\dots,N$. A partir disso, é possível calcular todas as derivadas em função dos pesos usando o método que descrevi, que é o Backpropagation.
É vidente que vários valores seriam usados mais de uma vez, então é necessário usar *memoização*. *Memoização* é uma técnica que consiste em
armazenar o resultado de uma operação para consultas futuras em vez de calcular o mesmo resultado várias vezes.

#### Economizando tempo

A princípio, cada iteração do Gradiente Descendente seria computada após calcular o vetor gradiente para *todos* os exemplos de treinamento.
Como podem haver muitos exemplos, calcular o gradiente para todos apenas para dar um passo pode demorar muito. Algo que é feito para economizar
tempo é dividir os exemplos de treinamento em grupos de mesmo tamanho, chamados *batches*. Em vez de fazer um passo do Gradiente Descendente após
passar por todo o conjunto de dados, o passo é feito após terminar de calcular os gradientes de cada batch. Isso causa algum ruído na descida do gradiente, porque, a cada batch, uma função diferente está sendo minimizada. Mas, no fim, as funções são muito próximas, então o método converge
ainda assim.

![](https://miro.medium.com/v2/resize:fit:908/1*bKSddSmLDaYszWllvQ3Z6A.png)

Imagem [dessa publicação](https://sweta-nit.medium.com/batch-mini-batch-and-stochastic-gradient-descent-e9bc4cacd461) do Medium.

Outra otimização que é feita é usar o que é chamado de *Gradiente Descendente Estocástico* (SGD, Stochastic Gradient Descent). Em vez de computar
todas as dimensões do vetor gradiente, apenas algumas dimensões aleatórias são computadas para serem otimizadas. Isso adiciona *bastante* ruído
na descida do gradiente, mas permite que mais iterações do gradiente descendente sejam calculadas em um espaço de tempo, e o método ainda
converge.

## Criando o gerador

Agora já temos todas as ferramentas para construir nosso gerador de lero-lero. Resolvi seguir o seguinte plano:

- Escolher o conjunto de dados a ser usado;
- Definir o vocabulário da rede neural;
- Gerar os dados de treinamento;
- Construir a rede neural;
- Treinar a rede neural com os dados de gerados;
- Fazer o programa que gera lero-lero usando a rede neural.

### Pré-requisitos

Usei algumas bibliotecas do Python na construção do gerador que você precisará instalar para acompanhar:

- `tensorflow_datasets`: Biblioteca que dá acesso aos datasets do catálogo do TensorFlow;
- `nltk`: Biblioteca de ferramentas de processamento de linguagem natural;
- `tensorflow`: Biblioteca de aprendizado de máquina.

Para instalá-los, basta executar:

    pip install tensorflow_datasets
    pip install nltk
    pip install tensorflow

### Conjunto de dados

Inicialmente, escolhi usar como conjunto de dados os artigos da Wikipedia, que estão disponíveis
[nesse dataset](https://huggingface.co/datasets/wikipedia) da Hugging Face, mas acabei mudando de ideia por dois motivos. O primeiro é que não
consegui baixar os dados, porque o link que o sistema mantinha do dump da Wikipedia estava desatualizado, então acontecia um erro na hora de baixar. O segundo motivo é que a quantidade de texto é *gigantesca* (20GB de texto), e usar essa quantidade de dados para treinar a rede gastaria
muito tempo.

Por fim, escolhi usar um conjunto de dados menor, que é um conjunto de dados de avaliações do IMDb, disponível
[nesse dataset](https://www.tensorflow.org/datasets/catalog/imdb_reviews) no catálogo do TensorFlow. Ele foi bem mais simples de conseguir usando
a biblioteca `tensorflow_datasets`, e era bem menor, mas também não era limitado. O único problema é que o lero-lero gerado seria parecido com
avaliações de filme. Mas, para um estudo, isso não seria um problema.

Note que escolhi usar a língua inglesa aqui. Isso porque ela é mais simples se tratando de quantidade de palavras distintas. Em português, temos
mais de vinte conjugações para o mesmo verbo, para diferentes pessoas e tempos verbais, e cada uma delas ocuparia um espaço no vocabulário. Em
inglês, cada verbo gera, no máximo, quatro variações. Como não queria criar um vocabulário muito grande, porque isso tornaria a rede neural maior
e, como consequência, mais lenta de treinar, optei pela língua inglesa.

Para importar o conjunto de dados, usei esse código:

In [2]:
import tensorflow_datasets as tfds

builder = tfds.load("imdb_reviews", data_dir="./data/", shuffle_files=True)
train_dataset = builder["train"]
test_dataset = builder["test"]

train_texts = [example["text"].numpy().decode("utf-8") for example in train_dataset]
test_texts = [example["text"].numpy().decode("utf-8") for example in test_dataset]

Esse código baixa o conjunto de dados no diretório `data`. Se ele já estiver baixado, ele usa os arquivos locais. Note que esse conjunto de dados
está dividido entre dados de treinamento e dados de teste. Isso me deu a ideia de manter alguns dados para teste, para ver a precisão da rede
neural em prever as palavras. No fim, isso não é tão importante, porque o problema que queremos resolver não é adivinhar palavras, e sim gerar
palavras.

### Vocabulário

Agora, eu teria que construir o vocabulário. Para fazer isso, seria necessário uma forma de separar o texto em palavras e, pelo menos em inglês,
isso não é uma tarefa tão simples quanto parece. A maneira mais ingênua de fazer isso é usando o método `split` das strings do Python, mas isso
causa alguns problemas.

In [82]:
print("I said: 'I don't like to take this path, it's scary.'".split(" "))

['I', 'said:', "'I", "don't", 'like', 'to', 'take', 'this', 'path,', "it's", "scary.'"]


Algumas palavras estão misturadas com as pontuações em volta, e não é isso que queremos. Poderíamos tirar a pontuação delas, mas isso causaria
alguns problemas. Por exemplo, não queremos tirar o apóstrofo de "don't", mas queremos tirar de "'I". Há muitos casos diferentes em que algo
imprevisto pode acontecer quando se trata de linguagem natural. Por isso, resolvi usar a biblioteca de processamento de linguagem natural
`nltk`, que possui algumas funções de *tokenização* (separação do texto em *tokens*).

In [83]:
from nltk import tokenize as tknz

tknz.casual_tokenize("I said: 'I don't like to take this path, it's scary.'")

['I',
 'said',
 ':',
 "'",
 'I',
 "don't",
 'like',
 'to',
 'take',
 'this',
 'path',
 ',',
 "it's",
 'scary',
 '.',
 "'"]

Testei algumas funções de tokenização, mas essa pareceu a função ideal.

Com isso, foi possível extrair o vocabulário e guardar no arquivo `data/vocabulary.txt`.

**Observação para poupar tempo:** Se você está interessado em usar o vocabulário que já foi gerado, pule o próximo código. O vocabulário que já
foi gerado está na pasta `data`. O código que vem depois desse carrega esse arquivo.

In [85]:
VOCABULARY_SIZE = 1000

word_count = {}

def is_word(token):
    token = token.lower()
    return token[0] >= 'a' and token[0] <= 'z'

for processed_text_count, text in enumerate(train_texts):
    for token in tknz.casual_tokenize(text):
        if not is_word(token): continue
        token = token.lower()
        if token in word_count.keys():
            word_count[token] += 1
        else:
            word_count[token] = 1
    print(f"Processed {processed_text_count+1}/{len(train_texts)}.")

count_word_list = [(count, word) for word, count in word_count.items()]
count_word_list = sorted(count_word_list, reverse=True)[:VOCABULARY_SIZE-2]
vocabulary = [word for count, word in count_word_list] + ["*", "_"]

with open("./data/vocabulary.txt", "w", encoding="utf-8") as vocabulary_file:
    for word in vocabulary:
        vocabulary_file.write(f"{word}\n")

print("Words:")
for count, word in count_word_list:
    print(f"    {word}: {count}")

Processed 1/25000.
Processed 2/25000.
Processed 3/25000.
Processed 4/25000.
Processed 5/25000.
Processed 6/25000.
Processed 7/25000.
Processed 8/25000.
Processed 9/25000.
Processed 10/25000.
Processed 11/25000.
Processed 12/25000.
Processed 13/25000.
Processed 14/25000.
Processed 15/25000.
Processed 16/25000.
Processed 17/25000.
Processed 18/25000.
Processed 19/25000.
Processed 20/25000.
Processed 21/25000.
Processed 22/25000.
Processed 23/25000.
Processed 24/25000.
Processed 25/25000.
Processed 26/25000.
Processed 27/25000.
Processed 28/25000.
Processed 29/25000.
Processed 30/25000.
Processed 31/25000.
Processed 32/25000.
Processed 33/25000.
Processed 34/25000.
Processed 35/25000.
Processed 36/25000.
Processed 37/25000.
Processed 38/25000.
Processed 39/25000.
Processed 40/25000.
Processed 41/25000.
Processed 42/25000.
Processed 43/25000.
Processed 44/25000.
Processed 45/25000.
Processed 46/25000.
Processed 47/25000.
Processed 48/25000.
Processed 49/25000.
Processed 50/25000.
Processed

Esse script lê todos os textos do conjunto de dados e ordena cada palavra distinta pelo número de ocorrências, mantendo apenas as 998 palavras
com mais ocorrências. Note que 2 palavras do vocabulário foram reservadas para dois casos especiais:

- `"*"`: Palavras desconhecidas. Essa é a palavra que daremos à rede neural quando aparecer uma palavra no contexto que não está no vocabulário;
- `"_"`: Ausência de palavra. Essa é a palavra que daremos à rede neural quando estivermos no início de uma frase, em que não há contexto.

Assim, construímos um vocabulário com 1000 palavras.

O código abaixo carrega o vocabulário do arquivo, e cria um mapeamento de palavras para índices e índices para palavras. Vamos usar esses
mapeamentos para gerar vetores one-hot para representar as palavras.

In [6]:
with open("./data/vocabulary.txt", "r", encoding="utf-8") as vocabulary_file:
    words = vocabulary_file.readlines()

words = [word[:-1] for word in words]

word_to_id_map = {word: id for id, word in enumerate(words)}
id_to_word = {id: word for id, word in enumerate(words)}

vocabulary = word_to_id_map.keys()
vocabulary_size = len(vocabulary)

def word_to_id(word):
    if word is None:
        word = "_"
    word = word.lower()
    if word not in word_to_id_map.keys():
        word = "*"
    return word_to_id_map[word]

print("Words:")
for id, word in id_to_word.items():
    print(f"    {id}: {word}")

Words:
    0: the
    1: and
    2: a
    3: of
    4: to
    5: is
    6: br
    7: in
    8: it
    9: i
    10: this
    11: that
    12: was
    13: as
    14: for
    15: with
    16: movie
    17: but
    18: film
    19: on
    20: not
    21: you
    22: are
    23: his
    24: have
    25: he
    26: be
    27: one
    28: all
    29: at
    30: by
    31: an
    32: they
    33: who
    34: from
    35: so
    36: like
    37: her
    38: or
    39: just
    40: about
    41: it's
    42: has
    43: if
    44: out
    45: some
    46: there
    47: what
    48: good
    49: more
    50: when
    51: very
    52: she
    53: even
    54: my
    55: no
    56: up
    57: time
    58: would
    59: which
    60: only
    61: story
    62: really
    63: their
    64: see
    65: had
    66: can
    67: were
    68: me
    69: than
    70: we
    71: well
    72: much
    73: been
    74: get
    75: will
    76: bad
    77: also
    78: into
    79: people
    80: other
    81:

### Dados de treinamento

Agora, geraremos os dados de treinamento para o modelo. Temos um conjunto de textos, mas o que realmente queremos usar para treinar a rede são
exemplos de pares contexto-palavra. Por exemplo, se temos a frase

> I don't like to go to the supermarket.

E queremos que os contextos tenham tamanho 2, então queremos gerar os seguintes dados:

> _ _ => I  
> _ I => don't  
> I don't => like  
> don't like => to  
> like to => go  
> to go => to  
> go to => the  
> to the => supermarket

Note que "supermarket" não está no vocabulário, e não é possível que a rede neural dê como resposta uma palavra que ela não conhece. Por isso,
removeremos os casos em que a palavra a predizer é desconhecida.

De qualquer forma, esse procedimento de gerar pares contexto-palavra será feito para todos os textos do conjunto de dados. Decidi fazer isso para
tamanhos de contexto diferentes, para ver como a saída do gerador de lero-lero é afetada.

**Observação para poupar tempo:** Se você está interessado em usar os dados de treino que já foram gerados, pule o próximo código. Os dados de
teste que usei já estão na pasta `data`.

In [31]:
import os

CONTEXT_SIZES = [1, 2, 3]

data_for_context_size = {context_size: [] for context_size in CONTEXT_SIZES}

for context_size in CONTEXT_SIZES:
    file_path = f"./data/context-to-word-{context_size}-train.txt"
    if os.path.exists(file_path):
        os.remove(file_path)

max_context_size = max(CONTEXT_SIZES)

for processed_text_count, text in enumerate(train_texts):

    for sentence in tknz.sent_tokenize(text):
        context = [word_to_id(None)] * max_context_size
        for token in tknz.casual_tokenize(sentence):
            if not is_word(token): continue
            word = token.lower()
            word_id = word_to_id(word)
            if word_id != word_to_id("*"):
                for context_size, data in data_for_context_size.items():
                    data.append((context[-context_size:], word_id))
            context.append(word_id)
    
    for context_size, data in data_for_context_size.items():
        for example in data:
            context, word = example
            with open(f"./data/context-to-word-{context_size}-train.txt", "a", encoding="utf-8") as data_file:
                row = " ".join(map(str, context + [word]))
                data_file.write(f"{row}\n")
        data.clear()

    print(f"Processed {processed_text_count+1}/{len(train_texts)}.")


Processed 1/25000.
Processed 2/25000.
Processed 3/25000.
Processed 4/25000.
Processed 5/25000.
Processed 6/25000.
Processed 7/25000.
Processed 8/25000.
Processed 9/25000.
Processed 10/25000.
Processed 11/25000.
Processed 12/25000.
Processed 13/25000.
Processed 14/25000.
Processed 15/25000.
Processed 16/25000.
Processed 17/25000.
Processed 18/25000.
Processed 19/25000.
Processed 20/25000.
Processed 21/25000.
Processed 22/25000.
Processed 23/25000.
Processed 24/25000.
Processed 25/25000.
Processed 26/25000.
Processed 27/25000.
Processed 28/25000.
Processed 29/25000.
Processed 30/25000.
Processed 31/25000.
Processed 32/25000.
Processed 33/25000.
Processed 34/25000.
Processed 35/25000.
Processed 36/25000.
Processed 37/25000.
Processed 38/25000.
Processed 39/25000.
Processed 40/25000.
Processed 41/25000.
Processed 42/25000.
Processed 43/25000.
Processed 44/25000.
Processed 45/25000.
Processed 46/25000.
Processed 47/25000.
Processed 48/25000.
Processed 49/25000.
Processed 50/25000.
Processed

Esse código usa a função `sent_tokenize` do `nltk` para separar os textos em frases, e usa o mapeamento gerado no código anterior para escrever
nos arquivos de saída os números que representam essas palavras.

Mas houve um pequeno problema quando vi o resultado gerado. Ele era grande demais! Apenas as 25 mil avaliações geraram mais de 4 milhões de linhas
em cada arquivo, o que tornaria o tempo de treinamento da rede neural muito longo. Por isso, editei os arquivos para deixá-los apenas com 10 mil
linhas.

### Criando a rede neural

Em vez de implementar a rede neural do zero, escolhi usar a biblioteca [Tensorflow](https://www.tensorflow.org/), esperando que fosse possível
usar todas as ferramentas que apresentei antes. E, adiantando, isso foi parcialmente possível. Meu melhor amigo na experimentação com o Tensorflow
foi [esse guia](https://towardsdatascience.com/deep-learning-with-python-neural-networks-complete-tutorial-6b53c0b06af0) do Medium.

Primeiro, tive que pensar na arquitetura da rede neural. Quantas camadas ocultas teria? Resolvi perguntar ao Google e encontrei
[essa resposta](https://stats.stackexchange.com/a/1097) no Stack Exchange. Resolvi então usar uma única camada oculta com o número de neurônios
igual à média do número de neurônios da entrada e o número de neurônios da saída. Se não desse certo, eu poderia testar com mais camadas ou mais
neurônios.

Mas quais funções de ativação eu usaria? Na série de Redes Neurais do 3Blue1Brown, a qual mencionei antes, ele trata de um problema de
classificação usando a função sigmóide. Isso porque ela limita os valores no intervalo $(0, 1)$, o que é ideal para classificações. Então resolvi
usar essa função na camada oculta. Quanto à função de ativação da camada de saída, resolvi usar a função softmax. Pelo que pesquisei, ela
geralmente é usada em problemas de classificação na camada de saída, porque faz com que todos os valores da saída estejam no intervalo $[0, 1]$ e
somem $1$.

Então, no fim, a rede neural seria composta por uma sequência de três camadas:

- Camada de entrada com $V \cdot C$ neurônios;
- Camada oculta com $\frac{V \cdot C + V}{2}$ neurônios, com função de ativação sigmóide;
- Camada de saída com $V$ neurônios, com função de ativação softmax.

Sendo $V$ o tamanho do vocabulário e $C$ o tamanho do contexto.

Para criar essa rede neural no Tensorflow, utilizei as classes de seu módulo `keras`. Crei cada camada individualmente. A camada de entrada é uma
instância da classe `Input`. Cada camada subsequente, em que cada neurônio é conectado a cada neurônio da camada anterior, é representada por uma
instância da classe `Dense`, inclusive a camada de saída. A essas camadas pode ser passado o nome suas funções de ativação, tornando esse processo
simples. A cada camada também é passada a camada anterior para ligar as duas. No fim, as camadas de entrada e de saída são passadas ao construtor
da classe `Model` para formar o modelo, que é a rede neural.

Também é necessário compilar cada modelo, escolhendo o método de otimização, a função erro e as métricas para medir a precisão da rede. Para o
método de otimização, escolhi o SGD (Gradiente Descendente Estocástico), e, para a função erro, escolhi a média do erro quadrático, que são as
ferramentas que já usávamos na disciplina. Para a métrica, escolhi usar a precisão categória. A precisão categórica é simplesmente a razão entre
predições corretas, em que a categoria com maior probabilidade era a categoria esperada, e o número de predições.

In [7]:
from tensorflow import keras as k

CONTEXT_SIZES = [1, 2, 3]

model_for_context_size = {}

for context_size in CONTEXT_SIZES:

    input_layer = k.layers.Input(
        name="input",
        shape=(vocabulary_size * context_size,)
    )

    hidden_layer = k.layers.Dense(
        name="hidden",
        units=(vocabulary_size * context_size + vocabulary_size) // 2,
        activation="sigmoid"
    )(input_layer)

    output_layer = k.layers.Dense(
        name="output",
        units=vocabulary_size,
        activation="softmax"
    )(hidden_layer)

    model = k.models.Model(
        name=f"model-{context_size}",
        inputs=input_layer,
        outputs=output_layer
    )

    model.summary()

    model.compile(
        optimizer="SGD",
        loss="mean_squared_error",
        metrics="categorical_accuracy"
    )

    model_for_context_size[context_size] = model

Model: "model-1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1000)]            0         
                                                                 
 hidden (Dense)              (None, 1000)              1001000   
                                                                 
 output (Dense)              (None, 1000)              1001000   
                                                                 
Total params: 2002000 (7.64 MB)
Trainable params: 2002000 (7.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________



Model: "model-2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 2000)]            0         
                                                                 
 hidden (Dense)              (None, 1500)              3001500   
                                                                 
 output (Dense)              (None, 1000)              1501000   
                                                                 
Total params: 4502500 (17.18 MB)
Trainable params: 4502500 (17.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "model-3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 3000)]            0         
                                                                 
 h

Antes de treinar o modelo, porém, precisamos fazer os nossos dados de treinamento legíveis para o modelo. Podemos fazer isso usando alguma classe
descendente da classe `Dataset` do Tensorflow. Como os dados que queremos estão em um arquivo de texto, a classe ideal para fazer isso é a classe
`TextLineDataset`. Para usá-la, precisei criar uma função para processar as linhas do arquivo, convertendo os números lidos em string para os
vetores one-hot, e passei essa fução ao objeto com o método `map`. Também usei o método `batch` para separar os dados em batches de 20 exemplos.

In [8]:
import tensorflow as tf

CONTEXT_SIZES = [1, 2, 3]
BATCH_SIZE = 20

dataset_for_context_size = {}

def parse_line(line, context_size):
    numbers = tf.strings.split(line, " ")
    output_context = tf.zeros(context_size * vocabulary_size)
    i = 0
    while i < len(numbers) - 1:
        number = numbers[i]
        final_index = tf.strings.to_number(number, tf.int32) + vocabulary_size*i
        output_context += tf.one_hot(final_index, vocabulary_size*context_size)
        i += 1
    output_word = tf.one_hot(tf.strings.to_number(numbers[-1], tf.int32), vocabulary_size)
    return (output_context, output_word)

for context_size in CONTEXT_SIZES:
    file_path = f"./data/context-to-word-{context_size}-train.txt"
    dataset = tf.data.TextLineDataset(file_path).map(lambda line: parse_line(line, context_size)).batch(BATCH_SIZE)
    dataset_for_context_size[context_size] = dataset


Por curiosidade, também gerei os arquivos de dados para os dados de teste do dataset para ver o quão bem o modelo iria predizer as palavras. O
código é muito similar ao usado para gerar os arquivos de dados de treinamento.

**Observação para poupar tempo:** Se você está interessado em usar os dados de teste que já foram gerados, pule o próximo código. Os dados de
teste que usei já estão na pasta `data`.

In [32]:
from nltk import tokenize as tknz
import os

CONTEXT_SIZES = [1, 2, 3]

test_data_for_context_size = {context_size: [] for context_size in CONTEXT_SIZES}

for context_size in CONTEXT_SIZES:
    file_path = f"./data/context-to-word-{context_size}-test.txt"
    if os.path.exists(file_path):
        os.remove(file_path)

max_context_size = max(CONTEXT_SIZES)

for processed_text_count, text in enumerate(test_texts):

    for sentence in tknz.sent_tokenize(text):
        context = [word_to_id(None)] * max_context_size
        for token in tknz.casual_tokenize(sentence):
            if not is_word(token): continue
            word = token.lower()
            word_id = word_to_id(word)
            if word_id != word_to_id("*"):
                for context_size, data in test_data_for_context_size.items():
                    data.append((context[-context_size:], word_id))
            context.append(word_id)
    
    for context_size, data in test_data_for_context_size.items():
        for example in data:
            context, word = example
            with open(f"./data/context-to-word-{context_size}-test.txt", "a", encoding="utf-8") as data_file:
                row = " ".join(map(str, context + [word]))
                data_file.write(f"{row}\n")
        data.clear()

    print(f"Processed {processed_text_count+1}/{len(train_texts)}.")


Processed 1/25000.
Processed 2/25000.
Processed 3/25000.
Processed 4/25000.
Processed 5/25000.
Processed 6/25000.
Processed 7/25000.
Processed 8/25000.
Processed 9/25000.
Processed 10/25000.
Processed 11/25000.
Processed 12/25000.
Processed 13/25000.
Processed 14/25000.
Processed 15/25000.
Processed 16/25000.
Processed 17/25000.
Processed 18/25000.
Processed 19/25000.
Processed 20/25000.
Processed 21/25000.
Processed 22/25000.
Processed 23/25000.
Processed 24/25000.
Processed 25/25000.
Processed 26/25000.
Processed 27/25000.
Processed 28/25000.
Processed 29/25000.
Processed 30/25000.
Processed 31/25000.
Processed 32/25000.
Processed 33/25000.
Processed 34/25000.
Processed 35/25000.
Processed 36/25000.
Processed 37/25000.
Processed 38/25000.
Processed 39/25000.
Processed 40/25000.
Processed 41/25000.
Processed 42/25000.
Processed 43/25000.
Processed 44/25000.
Processed 45/25000.
Processed 46/25000.
Processed 47/25000.
Processed 48/25000.
Processed 49/25000.
Processed 50/25000.
Processed

E esse é o código para criar os `TextLineDataset`s de teste.

In [10]:
import tensorflow as tf

CONTEXT_SIZES = [1, 2, 3]

test_dataset_for_context_size = {}

for context_size in CONTEXT_SIZES:
    file_path = f"./data/context-to-word-{context_size}-test.txt"
    dataset = tf.data.TextLineDataset(file_path).map(lambda line: parse_line(line, context_size)).batch(20)
    test_dataset_for_context_size[context_size] = dataset

Esse é o código que fiz para treinar o modelo. No TensorFlow, o treinamento é feito usando o método `fit` do modelo, passando um `Dataset` de
treinamento e um de teste. Também passamos o número de *epochs* que queremos que o treino dure. Uma *epoch* corresponde a uma passada no
conjunto de dados de treinamento inteiro. No final de cada epoch, o modelo é testado com o conjunto de dados de teste. Também podemos passar um
callback de "checkpoint", que é usado para salvar os pesos da rede neural no final de cada epoch.

Resolvi executar o código com um número pequeno de epochs e apenas para o tamanho de contexto 1 para ver o que acontece.

In [11]:
from tensorflow import keras as k

CONTEXT_SIZES_TO_TRAIN = [1]
EPOCHS = 15

for context_size in CONTEXT_SIZES_TO_TRAIN:

    model = model_for_context_size[context_size]

    checkpoint_callback = k.callbacks.ModelCheckpoint(
        filepath=f"./data/weights-{context_size}.ckpt",
        save_weights_only=True,
        verbose=1
    )

    model.fit(
        x=dataset_for_context_size[context_size],
        validation_data=test_dataset_for_context_size[context_size],
        epochs=EPOCHS,
        callbacks=[checkpoint_callback]
    )


Epoch 1/15



    490/Unknown - 3s 5ms/step - loss: 9.9923e-04 - categorical_accuracy: 0.0012
Epoch 1: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 4s 7ms/step - loss: 9.9923e-04 - categorical_accuracy: 0.0012 - val_loss: 9.9923e-04 - val_categorical_accuracy: 2.0000e-04
Epoch 2/15
495/500 [============================>.] - ETA: 0s - loss: 9.9923e-04 - categorical_accuracy: 0.0012
Epoch 2: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 3s 6ms/step - loss: 9.9923e-04 - categorical_accuracy: 0.0012 - val_loss: 9.9923e-04 - val_categorical_accuracy: 2.0000e-04
Epoch 3/15
498/500 [============================>.] - ETA: 0s - loss: 9.9923e-04 - categorical_accuracy: 0.0012
Epoch 3: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 3s 7ms/step - loss: 9.9923e-04 - categorical_accuracy: 0.0012 - val_loss: 9.9923e-04 - val_categorical_accuracy: 2.0000e-04
Epoch 4/15
500/500 [==============================] - 

Depois de 15 epochs, a precisão categórica no conjunto de dados de treinamento foi de 0.12%, e a precisão no conjunto de dados de teste foi de 0.02%. Embora a precisão estivesse aumentando a cada epoch, estava aumentando pouco demais para encontrarmos um resultado satisfatório
sem gastar tanto tempo. Resolvi então mexer na função erro dos modelos.

Resolvi usar a função erro entropia cruzada categórica, que é ideal para problemas de classificação. Ela é calculada dessa forma:

$$
E((x_1, \dots, x_T), (y_1, \dots, y_T)) = -\sum_{i = 1}^{T} y_1 \ln{x_1}
$$

Como $(y_1, \dots, y_T)$ é a saída esperada, que será um vetor one-hot, temos simplesmente que

$$
E((x_1, \dots, x_T), (y_1, \dots, y_T)) = -\ln{x_i}
$$

Sendo $i$ o natural tal que $y_i = 1$. Essa função busca penalizar mais predições, feitas para a categoria esperada, que estão distantes de $1$,
e menos as que estão mais próximas de $1$.

In [12]:
from tensorflow import keras as k

CONTEXT_SIZES_TO_TRAIN = [1]
EPOCHS = 15

for context_size, model in model_for_context_size.items():

    model.compile(
        optimizer="SGD",
        loss="categorical_crossentropy",
        metrics="categorical_accuracy"
    )

    model_for_context_size[context_size] = model

for context_size in CONTEXT_SIZES_TO_TRAIN:

    model = model_for_context_size[context_size]

    checkpoint_callback = k.callbacks.ModelCheckpoint(
        filepath=f"./data/weights-{context_size}.ckpt",
        save_weights_only=True,
        verbose=1
    )

    model.fit(
        x=dataset_for_context_size[context_size],
        validation_data=test_dataset_for_context_size[context_size],
        epochs=EPOCHS,
        callbacks=[checkpoint_callback]
    )


Epoch 1/15
    491/Unknown - 3s 5ms/step - loss: 5.6998 - categorical_accuracy: 0.0652
Epoch 1: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 4s 8ms/step - loss: 5.6969 - categorical_accuracy: 0.0658 - val_loss: 5.5392 - val_categorical_accuracy: 0.0698
Epoch 2/15
492/500 [============================>.] - ETA: 0s - loss: 5.4879 - categorical_accuracy: 0.0677
Epoch 2: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 4s 7ms/step - loss: 5.4880 - categorical_accuracy: 0.0680 - val_loss: 5.5125 - val_categorical_accuracy: 0.0698
Epoch 3/15
492/500 [============================>.] - ETA: 0s - loss: 5.4629 - categorical_accuracy: 0.0682
Epoch 3: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 3s 7ms/step - loss: 5.4633 - categorical_accuracy: 0.0685 - val_loss: 5.5090 - val_categorical_accuracy: 0.0698
Epoch 4/15
492/500 [============================>.] - ETA: 0s - loss: 5.4526 - categorical_

Houve uma melhora considerável. Agora, temos quase 6.97% de precisão nos dados de treino e 6.98% nos dados de teste. A precisão aumentou bem
devagar no final, porém. Conseguimos fazer algo melhor do que isso? O guia do Medium que eu estava seguindo menciona o otimizador Adam.
Pesquisando, descobri que esse algoritmo é uma modificação do SGD que, basicamente, combina duas modificações no Gradiente Descendente:

- O uso de "inércia" na descida do gradiente: Em vez de mover apenas o passo do gradiente, são adicionadas velocidade e aceleração ao
deslocamento. Para visualizar, imagine que o gráfico da função erro é uma superfície irregular, e a posição dos pesos atuais é uma bola sobre ela.
No Gradiente Descendente normal, essa bola se move sempre de forma a descer para a direção em que há mais inclinação para baixo, mas parando de
forma brusca. No Adam, e em algumas outras variações do Gradiente Descendente, essa bola possui inércia: quando ela desce por um declive, quando
chegar no fundo da depressão, ela continuará se movendo devido à aceleração que ganhou descendo o declive.

- A adaptação da taxa de aprendizado na descida do gradiente: Em vez de ter uma taxa de aprendizado fixa, a taxa de aprendizado é ajustada
dinamicamente de acordo com as últimas direções do gradiente. Isso é feito dividindo a taxa de aprendizado pela média das magnitudes de gradientes
recentes.

Resolvi então tentar usar o Adam.

In [13]:
from tensorflow import keras as k

CONTEXT_SIZES_TO_TRAIN = [1]
EPOCHS = 15

for context_size, model in model_for_context_size.items():

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics="categorical_accuracy"
    )

    model_for_context_size[context_size] = model

for context_size in CONTEXT_SIZES_TO_TRAIN:

    model = model_for_context_size[context_size]

    checkpoint_callback = k.callbacks.ModelCheckpoint(
        filepath=f"./data/weights-{context_size}.ckpt",
        save_weights_only=True,
        verbose=1
    )

    model.fit(
        x=dataset_for_context_size[context_size],
        validation_data=test_dataset_for_context_size[context_size],
        epochs=EPOCHS,
        callbacks=[checkpoint_callback]
    )


Epoch 1/15
    499/Unknown - 8s 15ms/step - loss: 6.1537 - categorical_accuracy: 0.0596
Epoch 1: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 9s 17ms/step - loss: 6.1528 - categorical_accuracy: 0.0598 - val_loss: 6.0111 - val_categorical_accuracy: 0.0802
Epoch 2/15
498/500 [============================>.] - ETA: 0s - loss: 5.7640 - categorical_accuracy: 0.0858
Epoch 2: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 8s 17ms/step - loss: 5.7636 - categorical_accuracy: 0.0859 - val_loss: 5.9182 - val_categorical_accuracy: 0.0860
Epoch 3/15
497/500 [============================>.] - ETA: 0s - loss: 5.5949 - categorical_accuracy: 0.1045
Epoch 3: saving model to ./data\weights-1.ckpt
500/500 [==============================] - 9s 17ms/step - loss: 5.5964 - categorical_accuracy: 0.1046 - val_loss: 5.7892 - val_categorical_accuracy: 0.0980
Epoch 4/15
498/500 [============================>.] - ETA: 0s - loss: 5.4274 - categori

Conseguimos 19.16% de precisão nos dados de treino e 12.78% nos dados de teste. Além disso, a precisão cresceu nos dados de teste, coisa que não
aconteceu nas outras tentativas.

Por fim, resolvi treinar os modelos usando o Adam, devido à melhor prefórmance.

**Observação para poupar tempo:** Se você está interessado em carregar os pesos que gerei, pule o próximo código. O código posterior carrega os
pesos já calculados.

In [ ]:
from tensorflow import keras as k

CONTEXT_SIZES_TO_TRAIN = [1, 2, 3]
EPOCHS = 300

for context_size in CONTEXT_SIZES_TO_TRAIN:

    model = model_for_context_size[context_size]

    checkpoint_callback = k.callbacks.ModelCheckpoint(
        filepath=f"./data/weights-{context_size}.ckpt",
        save_weights_only=True,
        verbose=1
    )

    model.fit(
        x=dataset_for_context_size[context_size],
        validation_data=test_dataset_for_context_size[context_size],
        epochs=EPOCHS,
        callbacks=[checkpoint_callback]
    )

Depois de deixar o treinamento acontecendo enquanto dormia, movi os dados do peso que foram salvos para a pasta `model`. Para carregar os pesos,
podemos usar o método `load_weights` do modelo. Também podemos avaliar a precisão deles novamente usando o método `evaluate` e passando os
conjuntos de dados.

In [14]:
CONTEXT_SIZES_TO_LOAD = [1, 2, 3]

for context_size in CONTEXT_SIZES_TO_LOAD:
    model = model_for_context_size[context_size]
    model.load_weights(f"./model/weights-{context_size}.ckpt")
    print(f"Loaded {model.name}.")
    print(f"For training dataset:")
    model.evaluate(dataset_for_context_size[context_size])
    print(f"For testing dataset:")
    model.evaluate(test_dataset_for_context_size[context_size])

Loaded model-1.
For training dataset:
500/500 [==============================] - 1s 3ms/step - loss: 3.1748 - categorical_accuracy: 0.2194
For testing dataset:
250/250 [==============================] - 1s 2ms/step - loss: 6.5925 - categorical_accuracy: 0.1316
Loaded model-2.
For training dataset:
500/500 [==============================] - 2s 3ms/step - loss: 1.4861 - categorical_accuracy: 0.5423
For testing dataset:
250/250 [==============================] - 1s 3ms/step - loss: 10.0700 - categorical_accuracy: 0.1216
Loaded model-3.
For training dataset:
500/500 [==============================] - 2s 4ms/step - loss: 0.7071 - categorical_accuracy: 0.7699
For testing dataset:
250/250 [==============================] - 1s 4ms/step - loss: 15.1850 - categorical_accuracy: 0.1050


Obtivemos enfim, depois de bastante treinamente, as seguintes precisões:

| Tamanho do contexto | Precisão para os dados de treino | Precisão para os dados de teste |
|---------------------|----------------------------------|---------------------------------|
| 1                   | 21.94%                           | 13.16%                          |
| 2                   | 54.23%                           | 12.16%                          |
| 3                   | 76.99%                           | 10.50%                          |

Curiosamente, a precisão para os dados de teste diminuiu conforme o tamanho do contexto aumentou, enquanto a precisão para os dados de treino
aumentou.

### Gerando o lero-lero

Com os modelos treinados, podemos enfim escrever o gerador de lero-lero que usa esses modelos.

O script abaixo gera o lero-lero progressivamente. Para gerar a próxima palavra, ele transforma as últimas palavras escolhidas em vetores one-hot
para servir de entrada para o modelo, que gera uma saída, que contém a probabilidade de cada palavra do vocabulário ser a próxima palavra. O
programa então sorteia a próxima palavra de acordo com essas probabilidades e insere na frase.

O usuário algum controle sobre o lero-lero sendo gerado. A cada passo do programa, o usuário pode inserir um dos seguintes comandos:

- `gen`: Gera a próxima palavra da frase.

- `gen [x]`: Gera as `x` próximas palavras da frase. Ex.: `gen 5`.

- `add [s]`: Adiciona `s` à frase. Ex.: `add going to the supermarket`.

- `stop`: Termina a frase e o programa.

Se o usuário inserir um comando desconhecido ou nenhum comando, o script simplesmente gera a próxima palavra.

In [ ]:
import random as rnd

CONTEXT_SIZE_TO_USE = 3

MODEL = model_for_context_size[CONTEXT_SIZE_TO_USE]

def make_prediction(context):
    model_input = tf.zeros(CONTEXT_SIZE_TO_USE * vocabulary_size)
    for i, word_id in enumerate(context):
        final_index = word_id + vocabulary_size*i
        model_input += tf.one_hot(final_index, depth=vocabulary_size*CONTEXT_SIZE_TO_USE)
    model_input = tf.reshape(model_input, (1, -1))
    prediction = MODEL(model_input)
    return prediction.numpy()[0]

def choose_word_id(prediction):
    return rnd.choices([i for i in range(vocabulary_size)], weights=prediction)[0]

NONE_ID = word_to_id(None)
context = [NONE_ID] * CONTEXT_SIZE_TO_USE
sentence = ""

def add_last_word(word, word_id=None):
    global context
    global sentence
    if word_id == None:
        word_id = word_to_id(word)
    if context[-1] == NONE_ID:
        word = word.capitalize()
    sentence = f"{sentence} {word}"
    context.append(word_id)
    context = context[1:]

def generate_last_word():
    prediction = make_prediction(context)
    word_id = choose_word_id(prediction)
    word = id_to_word[word_id]
    add_last_word(word, word_id)

def generate_last_words(count):
    for i in range(count):
        generate_last_word()

while True:
    
    print(f"Sentence: {sentence}")

    arguments = input("> ").split(" ")

    if arguments[0] == "stop":
        break

    elif arguments[0] == "gen":
        if len(arguments) == 1:
            generate_last_word()
        else:
            count = int(arguments[1])
            generate_last_words(count)
        continue

    elif arguments[0] == "add":
        words = arguments[1:]
        for word in words:
            add_last_word(word)
        continue

    generate_last_word()

sentence = f"{sentence}."
print(f"Sentence: {sentence}")


Aqui estão algumas frases que gerei que ficaram um pouco legais:

- Com tamanho do contexto 1:
    * "Makes this might have a for this film or i wish i remember every word."
    * "Why would be to their own right to get good while the late night and the role."
    * "So boring in this was killer as a small town which is a movie was able to happen."

- Com tamanho do contexto 2:
    * "This movie is an early movie years past her own age what a world of the day."
    * "I know that the movie on tv and it instead he takes the money is the one comment."
    * "While it is written and directed by the time and i think i just saw the movie so it should be my favorite."

- Com tamanho do contexto 3:
    * "Entertaining through and through from the beginning to the end if it was an first or second movie."
    * "It's basically the events of the film this is a big one for himself and a small one for the art."
    * "All said and done this is one good movie you have to sit through two hours."

## Considerações finais

Aqui seguem minhas considerações finais sobre esse trabalho.

### Sobre o resultado final

O gerador de lero-lero a que chegamos, ainda que não produza frases tão coesas, ainda é, de certa forma, impressionante levando em conta sua
simplicidade. É possível ver que as frases parecem melhor arquitetadas à medida que o tamanho do contexto aumenta. Mas há alguns problemas
notáveis.

O primeiro problema é que os textos do conjunto de dados estão cheios de "br". Não vi esse problema quando peguei alguns exemplos do dataset,
acredito se tratar do marcador de nova linha "&lt;br&gt;". Isso poderia ser resolvido simplesmente removendo esses marcadores dos dados. Por
enquanto, nosso gerador de lero-lero arrota de vez em quando.

O segundo problema é que, muitas vezes, o gerador quebra a sintaxe do inglês. Um exemplo é quando ele gera, por exemplo, "an first". Nesse
exemplo, a palavra "first" não poderia vir depois de "an", porque começa com consoante. Outro erro é quando um pronome é colocado onde se
espera um verbo. É natural que isso aconteça, já que o modelo não conhece as regras do inglês. Para corrigir isso, seria necessário criar um
programa ou modelos mais elaborados. A regra de "a"/"an", que é simples, poderia ser programada manualmente. Quanto à incompatibilidade de
classes gramaticais, poderiam ser usadas outros modelos que levam em conta as classes gramaticais das palavras dos contextos. Isso, porém,
demandaria a criação de outro classificador para atribuir classes gramaticais às palavras, e seria necessário um conjunto de dados com dados
desse tipo. Isso é um problema conhecido em processamento de linguagem natural; esses classificadores se chamam *POS taggers*. Uma parcela da
culpa por esse problema também vem do fato de que a palavra sorteada pode ser qualquer uma do vocabulário. Embora a chance de palavras inadequadas
serem escolhidas, no geral, seja menor, ainda é uma possibilidade que ocorre.

O terceiro problema é que, às vezes, o programa está gerando uma frase e, na próxima iteração, parece começar a gerar outra. Muitas dessas
ocorrências fariam sentido se uma vírgula fosse colocada entre as duas construções. O problema é que o modelo não foi feito para prever
qualquer tipo de pontuação. Muitas vezes, durante o treinamento, provavelmente duas orações separadas por uma vírgula se juntaram quando a
vírgula foi descartada, fazendo com que o modelo achasse que aquela era uma continuação normal de uma frase. Uma solução possível seria incluir
a vírgula no vocabulário do modelo.

Em suma, o modelo e a forma como ele é usado possuem alguns problemas que não puderam ser tratados no tempo deste trabalho. Há, porém, maneiras
conhecidas de melhorá-los para uma possível nova versão do gerador.

### Próximos passos

Partindo deste ponto, se fosse para continuar a pesquisa para melhorar o gerador de lero-lero, eu testaria para tamanhos de contexto maiores,
como 10 ou 15 palavras. Também tentaria usar vocabulários maiores, como 10000, dessa vez incluindo algumas pontuações menos problemáticas, como
a vírgula, o ponto-e-vírgula e os dois-pontos. Também acho que os arquivos de pares contexto-palavra usados no treinamento foram muito pequenos.
Com tempo suficiente, eu tentaria treinar os modelos com um 1000000 de exemplos em vez de apenas 10000, para ver se alguns padrões da língua
ficariam mais evidentes.

Outra coisa que eu tentaria fazer é um outro modelo que encontrei
[nessa publicação](https://medium.com/@codethulo/understanding-the-continuous-bag-of-words-cbow-model-architecture-working-mechanism-and-math-78c7284a8d5a)
do Medium. Consiste em, tendo um tamanho de contexto $C$, receber como entrada $C$ vetores one-hot separados, aos quais é aplicada uma
transformação por meio de uma camada oculta. Os vetores transformados seriam então somados, e a soma passaria por outra transformação, resultando
na saída da rede.

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*-Y44sz7jO9Yb_A0iivp9sg.png)

Imagem da publicação no Medium.

É possível construir essa rede usando TensorFlow. Na verdade, eu consegui fazer o código para isso:

In [34]:
from tensorflow import keras as k

CONTEXT_SIZE = 10
VOCABULARY_SIZE = 1000

input_layers = []
hidden_layers = []

for i in range(CONTEXT_SIZE):

    input_layer = k.layers.Input(
        name=f"input-{i}",
        shape=(VOCABULARY_SIZE,)
    )

    hidden_layer = k.layers.Dense(
        name=f"hidden-{i}",
        units=VOCABULARY_SIZE
    )(input_layer)

    input_layers.append(input_layer)
    hidden_layers.append(hidden_layer)

sum_layer = k.layers.Add(
    name="add"
)(hidden_layers)

output_layer = k.layers.Dense(
    name="output",
    units=VOCABULARY_SIZE
)(sum_layer)

model = k.models.Model(
    name=f"model-{CONTEXT_SIZE}",
    inputs=input_layers,
    outputs=output_layer
)

model.summary()

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics="categorical_accuracy"
)

Model: "model-10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input-0 (InputLayer)        [(None, 1000)]               0         []                            
                                                                                                  
 input-1 (InputLayer)        [(None, 1000)]               0         []                            
                                                                                                  
 input-2 (InputLayer)        [(None, 1000)]               0         []                            
                                                                                                  
 input-3 (InputLayer)        [(None, 1000)]               0         []                            
                                                                                           

O que eu não consegui fazer foi adaptar a função de processar as linhas do `TextLineDataset` para retornar várias entradas a serem usadas na
execução do método `fit` do modelo. Encontrar outra forma de prover esses dados ao modelo ou tentar novamente utilizando o `TextLineDataset`
seria um dos meus próximos passos.